# Polars Tutorial - Part 3: Data Manipulation

In this notebook, we'll explore advanced data manipulation techniques:
- Filtering and selecting with complex conditions
- Sorting and ranking
- Grouping and aggregations
- Joins and merges
- Window functions
- Data transformations

In [ ]:
import polars as pl
import os

DATA_DIR = '../data/'

# Load sample datasets
df_sales = pl.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))
df_employees = pl.read_json(os.path.join(DATA_DIR, 'employees.json'))

print("Sales Data:")
print(df_sales.head())
print("\nEmployee Data:")
print(df_employees.head())

## 1. Advanced Filtering

### 1.1 Multiple Conditions

In [ ]:
# Filter with multiple AND conditions
high_value_electronics = df_sales.filter(
    (pl.col('category') == 'Electronics') &
    (pl.col('revenue') > 1000)
)

print("High-value electronics sales:")
print(high_value_electronics)

# Filter with OR conditions
north_or_south = df_sales.filter(
    (pl.col('region') == 'North') | (pl.col('region') == 'South')
)

print("\nNorth or South regions:")
print(north_or_south)

### 1.2 Using is_in() for Multiple Values

In [ ]:
# Filter using is_in for multiple values
selected_regions = df_sales.filter(
    pl.col('region').is_in(['North', 'East'])
)

print("Sales in North and East regions:")
print(selected_regions)

# Negate with ~
not_electronics = df_sales.filter(
    ~pl.col('category').is_in(['Electronics'])
)

print("\nNon-electronics sales:")
print(not_electronics)

### 1.3 String Pattern Matching

In [ ]:
# Filter using string operations
desk_products = df_sales.filter(
    pl.col('product').str.contains('Desk')
)

print("Products containing 'Desk':")
print(desk_products)

# Case-insensitive matching
laptop_products = df_sales.filter(
    pl.col('product').str.to_lowercase().str.contains('laptop')
)

print("\nLaptop products (case-insensitive):")
print(laptop_products)

## 2. Selecting and Transforming Columns

### 2.1 Select with Transformations

In [ ]:
# Select and transform columns
df_transformed = df_sales.select([
    pl.col('product'),
    pl.col('quantity'),
    pl.col('price'),
    (pl.col('price') * pl.col('quantity')).alias('calculated_revenue'),
    (pl.col('price') * 0.9).alias('discounted_price')
])

print("Transformed columns:")
print(df_transformed.head())

### 2.2 Column Expressions with when-then-otherwise

In [ ]:
# Conditional column creation
df_categorized = df_sales.with_columns([
    pl.when(pl.col('revenue') > 2000)
      .then(pl.lit('High'))
      .when(pl.col('revenue') > 1000)
      .then(pl.lit('Medium'))
      .otherwise(pl.lit('Low'))
      .alias('revenue_category'),
    
    pl.when(pl.col('quantity') > 10)
      .then(pl.col('price') * 0.9)
      .otherwise(pl.col('price'))
      .alias('bulk_price')
])

print("Categorized data:")
print(df_categorized.select(['product', 'revenue', 'revenue_category', 'quantity', 'bulk_price']).head())

## 3. Sorting and Ranking

### 3.1 Advanced Sorting

In [ ]:
# Sort by multiple columns with different orders
df_sorted = df_sales.sort(
    ['category', 'revenue'],
    descending=[False, True]
)

print("Sorted by category (asc) then revenue (desc):")
print(df_sorted.select(['category', 'product', 'revenue']))

# Top N items
top_5_revenue = df_sales.sort('revenue', descending=True).head(5)
print("\nTop 5 by revenue:")
print(top_5_revenue.select(['product', 'revenue']))

### 3.2 Ranking

In [ ]:
# Add rank column
df_ranked = df_sales.with_columns([
    pl.col('revenue').rank(method='dense', descending=True).alias('revenue_rank'),
    pl.col('quantity').rank(method='ordinal').alias('quantity_rank')
])

print("Data with rankings:")
print(df_ranked.select(['product', 'revenue', 'revenue_rank', 'quantity', 'quantity_rank']).head(10))

## 4. Grouping and Aggregations

### 4.1 Basic Group By

In [ ]:
# Group by category and aggregate
category_stats = df_sales.group_by('category').agg([
    pl.count('product').alias('num_transactions'),
    pl.sum('revenue').alias('total_revenue'),
    pl.mean('revenue').alias('avg_revenue'),
    pl.sum('quantity').alias('total_quantity')
]).sort('total_revenue', descending=True)

print("Sales by category:")
print(category_stats)

### 4.2 Multiple Group By

In [ ]:
# Group by multiple columns
region_category_stats = df_sales.group_by(['region', 'category']).agg([
    pl.sum('revenue').alias('total_revenue'),
    pl.mean('price').alias('avg_price'),
    pl.count().alias('count')
]).sort(['region', 'total_revenue'], descending=[False, True])

print("Sales by region and category:")
print(region_category_stats)

### 4.3 Advanced Aggregations

In [ ]:
# Multiple aggregations on same column
product_stats = df_sales.group_by('product').agg([
    pl.count().alias('times_sold'),
    pl.sum('quantity').alias('total_quantity'),
    pl.sum('revenue').alias('total_revenue'),
    pl.min('price').alias('min_price'),
    pl.max('price').alias('max_price'),
    pl.mean('price').alias('avg_price'),
    pl.std('price').alias('price_std')
]).sort('total_revenue', descending=True)

print("Detailed product statistics:")
print(product_stats)

### 4.4 Group By with Filtering

In [ ]:
# Filter groups after aggregation
high_volume_regions = df_sales.group_by('region').agg([
    pl.sum('revenue').alias('total_revenue'),
    pl.count().alias('num_sales')
]).filter(pl.col('num_sales') > 2)

print("Regions with more than 2 sales:")
print(high_volume_regions)

## 5. Joining DataFrames

### 5.1 Creating a Second DataFrame for Joins

In [ ]:
# Create a regions DataFrame
df_regions = pl.DataFrame({
    'region': ['North', 'South', 'East', 'West'],
    'manager': ['Alice Johnson', 'Bob Smith', 'Charlie Davis', 'Diana Martinez'],
    'office_city': ['New York', 'Miami', 'Boston', 'Seattle']
})

print("Regions DataFrame:")
print(df_regions)

### 5.2 Inner Join

In [ ]:
# Inner join
df_joined = df_sales.join(df_regions, on='region', how='inner')

print("Inner join result:")
print(df_joined.select(['product', 'region', 'manager', 'office_city', 'revenue']).head())

### 5.3 Left Join

In [ ]:
# Left join
df_left = df_sales.join(df_regions, on='region', how='left')

print("Left join result:")
print(df_left.select(['product', 'region', 'manager', 'revenue']).head())
print(f"Rows in result: {df_left.height}")

### 5.4 Join with Different Column Names

In [ ]:
# Create customer DataFrame
df_customers = pl.DataFrame({
    'cust_id': ['C001', 'C002', 'C003', 'C004', 'C005'],
    'customer_name': ['Acme Corp', 'Tech Inc', 'Global Ltd', 'Mega Corp', 'Super Co'],
    'industry': ['Manufacturing', 'Technology', 'Finance', 'Retail', 'Healthcare']
})

# Join on different column names
df_with_customers = df_sales.join(
    df_customers,
    left_on='customer_id',
    right_on='cust_id',
    how='left'
)

print("Join with different column names:")
print(df_with_customers.select(['product', 'customer_id', 'customer_name', 'industry', 'revenue']).head())

## 6. Window Functions

### 6.1 Cumulative Sum

In [ ]:
# Calculate cumulative sum
df_cumsum = df_sales.sort('date').with_columns([
    pl.col('revenue').cum_sum().alias('cumulative_revenue'),
    pl.col('quantity').cum_sum().alias('cumulative_quantity')
])

print("Cumulative sums:")
print(df_cumsum.select(['date', 'product', 'revenue', 'cumulative_revenue']).head(10))

### 6.2 Window Functions with over()

In [ ]:
# Calculate running statistics per category
df_window = df_sales.with_columns([
    pl.col('revenue').sum().over('category').alias('category_total_revenue'),
    pl.col('revenue').mean().over('category').alias('category_avg_revenue'),
    pl.col('revenue').max().over('category').alias('category_max_revenue')
])

print("Window functions by category:")
print(df_window.select([
    'category', 'product', 'revenue', 
    'category_total_revenue', 'category_avg_revenue'
]).head(10))

### 6.3 Ranking Within Groups

In [ ]:
# Rank products within each category
df_ranked_category = df_sales.with_columns([
    pl.col('revenue').rank(method='dense', descending=True).over('category').alias('rank_in_category')
])

print("Ranked within categories:")
print(df_ranked_category.select([
    'category', 'product', 'revenue', 'rank_in_category'
]).sort(['category', 'rank_in_category']))

### 6.4 Rolling Windows

In [ ]:
# Calculate rolling averages
df_rolling = df_sales.sort('date').with_columns([
    pl.col('revenue').rolling_mean(window_size=3).alias('revenue_3day_avg'),
    pl.col('quantity').rolling_sum(window_size=3).alias('quantity_3day_sum')
])

print("Rolling statistics:")
print(df_rolling.select(['date', 'revenue', 'revenue_3day_avg', 'quantity', 'quantity_3day_sum']).head(10))

## 7. Pivoting and Unpivoting

### 7.1 Pivot Table

In [ ]:
# Create pivot table
pivot_table = df_sales.pivot(
    values='revenue',
    index='product',
    columns='region',
    aggregate_function='sum'
)

print("Pivot table - Revenue by Product and Region:")
print(pivot_table)

### 7.2 Melt (Unpivot)

In [ ]:
# Create a wide DataFrame for demonstration
df_wide = pl.DataFrame({
    'student': ['Alice', 'Bob', 'Charlie'],
    'math': [85, 90, 78],
    'science': [88, 85, 92],
    'english': [90, 88, 85]
})

print("Wide format:")
print(df_wide)

# Melt to long format
df_long = df_wide.melt(
    id_vars='student',
    value_vars=['math', 'science', 'english'],
    variable_name='subject',
    value_name='score'
)

print("\nLong format:")
print(df_long)

## 8. String Operations

In [ ]:
# Various string operations
df_strings = df_sales.with_columns([
    pl.col('product').str.to_uppercase().alias('product_upper'),
    pl.col('product').str.to_lowercase().alias('product_lower'),
    pl.col('product').str.len_chars().alias('product_length'),
    pl.col('region').str.slice(0, 2).alias('region_abbrev')
])

print("String operations:")
print(df_strings.select(['product', 'product_upper', 'product_length', 'region', 'region_abbrev']).head())

## 9. Date and Time Operations

In [ ]:
# Parse dates and extract components
df_dates = df_sales.with_columns([
    pl.col('date').str.strptime(pl.Date, format='%Y-%m-%d').alias('parsed_date')
]).with_columns([
    pl.col('parsed_date').dt.year().alias('year'),
    pl.col('parsed_date').dt.month().alias('month'),
    pl.col('parsed_date').dt.day().alias('day'),
    pl.col('parsed_date').dt.weekday().alias('weekday')
])

print("Date operations:")
print(df_dates.select(['date', 'parsed_date', 'year', 'month', 'day', 'weekday']).head())

## 10. Summary

In this notebook, we explored:
- ✅ Advanced filtering techniques
- ✅ Column transformations and conditional logic
- ✅ Sorting and ranking
- ✅ Grouping and aggregations
- ✅ Joining DataFrames
- ✅ Window functions
- ✅ Pivoting and melting
- ✅ String and date operations

### Key Takeaways:
1. Polars uses expression syntax with `pl.col()` for powerful transformations
2. Window functions allow calculations within groups without reducing rows
3. The API is designed for method chaining
4. All operations are optimized and run in parallel when possible

**Next:** In the next notebook, we'll create visualizations from our data!